In [24]:
import pandas as pd
import numpy as np



## 载入之前清洗好的数据数据

In [4]:
train = pd.read_csv('../data/train_clean.csv')

In [6]:
train.head()

,label,text,token
0,1,sweet united nations video just in time for c...,"['sweet', 'united', 'nation', 'video', 'time',..."
1,1,we are rumored to have talked to erv's agent ...,"['rumored', 'talked', 'erv', 'agent', 'angel',..."
2,1,hey there! nice to see you minnesota nd winter...,"['hey', 'nice', 'see', 'minnesota', 'nd', 'win..."
3,0,3 episodes left i'm dying over here,"['3', 'episode', 'left', 'dying']"
4,1,i can't breathe! was chosen as the most notabl...,"['breathe', 'chosen', 'notable', 'quote', 'yea..."


## 划分训练集测试集提取文本特征 

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train['token'], train['label'], test_size=0.2, random_state=40)

vectorizer = CountVectorizer(max_features=5000)
vectorizer.fit(X_train, y_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [39]:
X_train[6104], X_test[0]

(<1x5000 sparse matrix of type '<class 'numpy.int64'>'
 	with 4 stored elements in Compressed Sparse Row format>,
 <1x5000 sparse matrix of type '<class 'numpy.int64'>'
 	with 9 stored elements in Compressed Sparse Row format>)

## 使用朴素贝叶斯进行构建分类器，朴素贝叶斯理论细节参考博客https://blog.csdn.net/han_xiaoyang/article/details/50616559

In [18]:
from sklearn.naive_bayes import MultinomialNB

#载入模型
classifier = MultinomialNB()
#模型训练
classifier.fit(X_train, y_train.tolist())
#预测结果
classifier.score(X_test, y_test.tolist())

0.7373935821872953

## 使用SVM模型进行文本分类，SVM理论细节参考李航的统计学习方法

In [21]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(X_train, y_train.tolist())
accuracy_score(clf.predict(X_test), y_test.tolist())

0.4865749836280288

## 加入LDA特征

In [23]:
from sklearn.decomposition import LatentDirichletAllocation 
lda = LatentDirichletAllocation(n_components=30, 
                                max_iter=50,
                                learning_method='batch')
train_lda = lda.fit_transform(X_train)
test_lda = lda.transform(X_test)

C:\Anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [33]:
train_lda[0], test_lda[0], len(train_lda)

(array([0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.87916667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667,
        0.00416667, 0.00416667, 0.00416667, 0.00416667, 0.00416667]),
 array([0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
        0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
        0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
        0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
        0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
        0.00333333, 0.00333333, 0.90333333, 0.00333333, 0.00333333]),
 6105)

In [40]:
np.array(X_train[0])

array(<1x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>, dtype=object)

In [31]:
X_train = np.vstack((np.array(X_train), np.array(train_lda)))
# X_test = np.hstack((X_test, test_lda))

ValueError: all the input array dimensions except for the concatenation axis must match exactly